In [0]:
%python
import numpy as np # linear algebra
import pandas as pd 

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import sklearn
import random
import os

from pyspark.sql import SparkSession 
from pyspark.ml  import Pipeline     
from pyspark.sql import SQLContext  
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer

In [0]:
%python
spark = SparkSession.builder.appName('IPL').getOrCreate()

In [0]:
%python
df =  spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/girivashini21@gmail.com/IPL_Matches_2008_2020-1.csv")

df1 = spark.read.format("csv").option("header","true").load("dbfs:/FileStore/shared_uploads/girivashini21@gmail.com/IPL_Ball_by_Ball_2008_2020-3.csv")


df1.limit(3).toPandas()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NA,NA,NA,NA,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NA,NA,NA,NA,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NA,NA,NA,NA,Kolkata Knight Riders,Royal Challengers Bangalore


In [0]:
df.limit(3).toPandas()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140,N,NA,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33,N,NA,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9,N,NA,Aleem Dar,GA Pratapkumar


In [0]:
%python
print('Data overview')
df.printSchema()


print('Columns overview')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])




print('Data overview')
df1.printSchema()


print('Columns overview')
pd.DataFrame(df1.dtypes, columns = ['Column Name','Data type'])

Data overview
root
 |-- id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- neutral_venue: string (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- result: string (nullable = true)
 |-- result_margin: string (nullable = true)
 |-- eliminator: string (nullable = true)
 |-- method: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)

Columns overview
Data overview
root
 |-- id: string (nullable = true)
 |-- inning: string (nullable = true)
 |-- over: string (nullable = true)
 |-- ball: string (nullable = true)
 |-- batsman: string (nullable = true)
 |-- non_striker: string (nullable = true)
 |-- bowler: string (nullable = true)
 |--

,Column Name,Data type
0,id,string
1,inning,string
2,over,string
3,ball,string
4,batsman,string
5,non_striker,string
6,bowler,string
7,batsman_runs,string
8,extra_runs,string
9,total_runs,string


In [0]:
# Data Preparation and Cleaning
# Lets convert some columns to integer typefrom pyspark.sql.types import IntegerType

from pyspark.sql.types import IntegerType
df1 = df1.withColumn("is_wicket", df1["is_wicket"].cast(IntegerType()))
df1 = df1.withColumn("batsman_runs", df1["batsman_runs"].cast(IntegerType()))
df1 = df1.withColumn("extra_runs", df1["extra_runs"].cast(IntegerType()))
df1 = df1.withColumn("total_runs", df1["total_runs"].cast(IntegerType()))

In [0]:
df1.summary().show(vertical=True)

-RECORD 0--------------------------------
 summary          | count                
 id               | 193468               
 inning           | 193468               
 over             | 193468               
 ball             | 193468               
 batsman          | 193468               
 non_striker      | 193468               
 bowler           | 193468               
 batsman_runs     | 193468               
 extra_runs       | 193468               
 total_runs       | 193468               
 non_boundary     | 193468               
 is_wicket        | 193468               
 dismissal_kind   | 193468               
 player_dismissed | 193468               
 fielder          | 193468               
 extras_type      | 193468               
 batting_team     | 193468               
 bowling_team     | 193468               
-RECORD 1--------------------------------
 summary          | mean                 
 id               | 756768.8084386048    
 inning           | 1.482131411913

In [0]:
print('Data frame describe (string and numeric columns only):')
df.describe().toPandas()

print(f'There are total {df.count()} row, Let print first 2 data rows:')
df.limit(2).toPandas()



print('Data frame describe (string and numeric columns only):')
df1.describe().toPandas()

print(f'There are total {df1.count()} row, Let print first 2 data rows:')
df1.limit(2).toPandas()



Data frame describe (string and numeric columns only):
There are total 816 row, Let print first 2 data rows:
Data frame describe (string and numeric columns only):
There are total 193468 row, Let print first 2 data rows:


,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NA,NA,NA,NA,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NA,NA,NA,NA,Kolkata Knight Riders,Royal Challengers Bangalore


In [0]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show(vertical =True)

df1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df1.columns]).show(vertical =True)

-RECORD 0--------------
 id              | 0   
 city            | 0   
 date            | 0   
 player_of_match | 0   
 venue           | 0   
 neutral_venue   | 0   
 team1           | 0   
 team2           | 0   
 toss_winner     | 0   
 toss_decision   | 0   
 winner          | 0   
 result          | 0   
 result_margin   | 0   
 eliminator      | 0   
 method          | 0   
 umpire1         | 0   
 umpire2         | 0   

-RECORD 0---------------
 id               | 0   
 inning           | 0   
 over             | 0   
 ball             | 0   
 batsman          | 0   
 non_striker      | 0   
 bowler           | 0   
 batsman_runs     | 0   
 extra_runs       | 0   
 total_runs       | 0   
 non_boundary     | 0   
 is_wicket        | 0   
 dismissal_kind   | 0   
 player_dismissed | 0   
 fielder          | 0   
 extras_type      | 0   
 batting_team     | 0   
 bowling_team     | 0   



In [0]:
print(df.columns)
print(df1.columns)

['id', 'city', 'date', 'player_of_match', 'venue', 'neutral_venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'result', 'result_margin', 'eliminator', 'method', 'umpire1', 'umpire2']
['id', 'inning', 'over', 'ball', 'batsman', 'non_striker', 'bowler', 'batsman_runs', 'extra_runs', 'total_runs', 'non_boundary', 'is_wicket', 'dismissal_kind', 'player_dismissed', 'fielder', 'extras_type', 'batting_team', 'bowling_team']


In [0]:
# Which player have most sixes in IPL

six=df1.filter(df1.batsman_runs=='6')
six.groupBy('batsman').count().sort('count',ascending=False).show(10)

+--------------+-----+
|       batsman|count|
+--------------+-----+
|      CH Gayle|  349|
|AB de Villiers|  235|
|      MS Dhoni|  216|
|     RG Sharma|  214|
|       V Kohli|  202|
|    KA Pollard|  198|
|     DA Warner|  195|
|      SK Raina|  194|
|     SR Watson|  190|
|    RV Uthappa|  163|
+--------------+-----+
only showing top 10 rows



In [0]:
#Which bowler has most wickets in power player overs in ipl?

power=df1 [ df1['over'].between(0,5) ]

#filtering out obstructing the field and run out dismissal kind 
power_wicket_taker=power[ ~power['dismissal_kind'].isin ([ 'obstructing the field' , 'run out' ]) ]

power_wicket_taker.groupBy('bowler').sum('is_wicket').sort('sum(is_wicket)',ascending=False).show()

+--------------+--------------+
|        bowler|sum(is_wicket)|
+--------------+--------------+
|        Z Khan|            53|
|Sandeep Sharma|            53|
|       B Kumar|            48|
|      UT Yadav|            45|
|   DS Kulkarni|            44|
|      I Sharma|            43|
|      R Ashwin|            42|
|       P Kumar|            40|
|       A Nehra|            40|
|    SL Malinga|            37|
|      M Morkel|            36|
|      RP Singh|            35|
|     MM Sharma|            35|
|      DW Steyn|            34|
|     SR Watson|            34|
|     JA Morkel|            34|
|     DL Chahar|            33|
|      AB Dinda|            33|
|MJ McClenaghan|            31|
| R Vinay Kumar|            30|
+--------------+--------------+
only showing top 20 rows



In [0]:
#Which bowler has most 5 wickets haul in ipl?

fv = df1[ ~df1['dismissal_kind'].isin('obstructing the field', 'run out' , 'retired hurt') ]

five=fv.groupby(['id','bowler']).sum('is_wicket')

five_wickets =  five [ (five['sum(is_wicket)'] >= 5) ]

five_wickets.groupby('bowler').count().sort('count',ascending=False).show(10)

+--------------+-----+
|        bowler|count|
+--------------+-----+
|    JD Unadkat|    2|
|   JP Faulkner|    2|
|       B Kumar|    1|
|     SP Narine|    1|
|    SL Malinga|    1|
|        AJ Tye|    1|
|AD Mascarenhas|    1|
|      A Kumble|    1|
|      A Mishra|    1|
|       A Zampa|    1|
+--------------+-----+
only showing top 10 rows



In [0]:
#Counting number of times a player is dismissed

player_dismissed =  df1.groupby('player_dismissed').count()

total_runs  =  df1.groupBy('batsman').sum('batsman_runs')

joined_data =  player_dismissed.join(total_runs, player_dismissed.player_dismissed == total_runs.batsman, 'inner') 

#Dividing runs by number of times they have been dismissed
result_7   =   joined_data.withColumn("Batsmen Avg", col("sum(batsman_runs)")/col("count")).sort('Batsmen Avg',ascending=False)


result_7.select('batsman','Batsmen Avg').show(10)

+-------------+------------------+
|      batsman|       Batsmen Avg|
+-------------+------------------+
|   MN van Wyk|55.666666666666664|
|   RD Gaikwad|              51.0|
|     AC Voges|             45.25|
|     KL Rahul| 44.86440677966102|
|      HM Amla| 44.38461538461539|
|Iqbal Abdulla|              44.0|
|    DA Warner| 42.71544715447155|
|  JM Bairstow|41.578947368421055|
|     CH Gayle| 41.13793103448276|
|     MS Dhoni|40.991150442477874|
+-------------+------------------+
only showing top 10 rows



In [0]:
join_main=df1.join(df,on='id')
join_main.limit(3).toPandas()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,...,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,...,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140,N,NA,Asad Rauf,RE Koertzen
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,...,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140,N,NA,Asad Rauf,RE Koertzen
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,...,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140,N,NA,Asad Rauf,RE Koertzen


In [0]:
#Which Player has scored most runs in a particular year?
sum_bat=join_main.groupby(['Batsman','date']).sum('batsman_runs').sort("sum(batsman_runs)",ascending=False)
sum_bat.show()

+--------------+----------+-----------------+
|       Batsman|      date|sum(batsman_runs)|
+--------------+----------+-----------------+
|      CH Gayle|2013-04-23|              175|
|   BB McCullum|2008-04-18|              158|
|AB de Villiers|2015-05-10|              133|
|      KL Rahul|2020-09-24|              132|
|AB de Villiers|2016-05-14|              129|
|       RR Pant|2018-05-10|              128|
|      CH Gayle|2012-05-17|              128|
|       M Vijay|2010-04-03|              127|
|     DA Warner|2017-04-30|              126|
|      V Sehwag|2014-05-30|              122|
|   PC Valthaty|2011-04-13|              120|
|      V Sehwag|2011-05-05|              119|
|      CH Gayle|2015-05-06|              117|
|     SR Watson|2018-05-27|              117|
|     A Symonds|2008-04-24|              117|
|    MEK Hussey|2008-04-19|              116|
|      SE Marsh|2008-05-28|              115|
|       WP Saha|2014-06-01|              115|
|   JM Bairstow|2019-03-31|       

In [0]:
#finding all matches which were effected by D/L method
DL=join_main[join_main['method']=='D/L']

#finding out total runs scored by every batsman in each match effected by D/L 
sum1=DL.groupBy(['id','batsman']).sum('batsman_runs')

from pyspark.sql.window import Window
from pyspark.sql.functions import rank
from pyspark.sql.functions import col


window   =   Window.partitionBy('id').orderBy(col('sum(batsman_runs)').desc())
res_4   =   sum1.withColumn('rank',rank().over(window))
res_4.filter(res_4.rank==1).show()

+-------+------------+-----------------+----+
|     id|     batsman|sum(batsman_runs)|rank|
+-------+------------+-----------------+----+
|1082648|   DA Warner|               37|   1|
|1136566|   AM Rahane|               45|   1|
|1136578|     CA Lynn|               74|   1|
|1136592|     RR Pant|               69|   1|
| 336022|    V Sehwag|               51|   1|
| 336025| Salman Butt|               73|   1|
| 392183|    V Sehwag|               38|   1|
| 392183|      K Goel|               38|   1|
| 392186|    CH Gayle|               44|   1|
| 392214|   ML Hayden|               89|   1|
| 501215|    SK Raina|               50|   1|
| 501245| S Badrinath|               54|   1|
| 501255|    CH Gayle|               38|   1|
| 733993|  KD Karthik|               39|   1|
| 829743|   DA Warner|               91|   1|
| 829807|MC Henriques|               57|   1|
| 980943|    S Dhawan|               56|   1|
| 980989|   YK Pathan|               37|   1|
| 980997|   AM Rahane|            

In [0]:
# Query to get a list of top 10 players with the highest batting average

#Counting number of times a player is dismissed
player_dismissed=df1.groupby('player_dismissed').count()

total_runs=df1.groupBy('batsman').sum('batsman_runs')

joined_data=player_dismissed.join(total_runs, player_dismissed.player_dismissed == total_runs.batsman, 'inner') 

#Dividing runs by number of times they have been dismissed
result_7=joined_data.withColumn( "Batsmen Avg", col("sum(batsman_runs)")  /  col("count") ).sort('Batsmen Avg',ascending=False )

result_7.select('batsman','Batsmen Avg').show(10)

+-------------+------------------+
|      batsman|       Batsmen Avg|
+-------------+------------------+
|   MN van Wyk|55.666666666666664|
|   RD Gaikwad|              51.0|
|     AC Voges|             45.25|
|     KL Rahul| 44.86440677966102|
|      HM Amla| 44.38461538461539|
|Iqbal Abdulla|              44.0|
|    DA Warner| 42.71544715447155|
|  JM Bairstow|41.578947368421055|
|     CH Gayle| 41.13793103448276|
|     MS Dhoni|40.991150442477874|
+-------------+------------------+
only showing top 10 rows



In [0]:
#Which Player has taken most wickets in a particular year?

sum_wic=join_main.groupby(['bowler','date']).sum('is_wicket').sort("sum(is_wicket)",ascending=False)
sum_wic.show()

+--------------+----------+--------------+
|        bowler|      date|sum(is_wicket)|
+--------------+----------+--------------+
|       A Zampa|2016-05-10|             6|
| Sohail Tanvir|2008-05-04|             6|
|    AD Russell|2016-05-04|             6|
|     DJG Sammy|2013-05-11|             6|
|     AS Joseph|2019-04-06|             6|
|  CRD Fernando|2008-05-28|             5|
|      RP Singh|2009-04-19|             5|
| R Vinay Kumar|2010-03-23|             5|
|     VY Mahesh|2008-05-24|             5|
|      A Kumble|2010-03-18|             5|
|     SP Narine|2012-05-07|             5|
|      MM Patel|2011-05-10|             5|
|     RA Jadeja|2012-04-07|             5|
|      R Ashwin|2012-05-25|             5|
|     SP Narine|2013-04-03|             5|
|     SP Narine|2012-04-15|             5|
|    JD Unadkat|2013-05-10|             5|
|      A Kumble|2009-04-18|             5|
|AD Mascarenhas|2012-04-12|             5|
|      BJ Hodge|2011-05-15|             5|
+----------

In [0]:
most_year=join_main.groupby(['date','id','batsman']).sum('batsman_runs')
most_hun=most_year[most_year['sum(batsman_runs)'] >= 100]
most_hun.groupby(['batsman','date']).count().sort('count',ascending=False).show(10)

+---------+----------+-----+
|  batsman|      date|count|
+---------+----------+-----+
|  HM Amla|2017-04-20|    1|
|SR Watson|2015-05-16|    1|
|DA Warner|2019-03-31|    1|
|  HM Amla|2017-05-07|    1|
| CH Gayle|2012-05-17|    1|
|DA Warner|2017-04-30|    1|
| V Sehwag|2014-05-30|    1|
|AM Rahane|2019-04-22|    1|
|SR Watson|2018-05-27|    1|
|  V Kohli|2016-04-24|    1|
+---------+----------+-----+
only showing top 10 rows



In [0]:
most_duck = join_main.groupby(['date','id','batsman']).sum('batsman_runs')

most_duck_year = most_duck[most_duck['sum(batsman_runs)']  == 0]

most_duck_year.groupby(['batsman','date']).count().sort('count',ascending = False).show(20)

+-----------------+----------+-----+
|          batsman|      date|count|
+-----------------+----------+-----+
|          PR Shah|2008-05-24|    1|
|     Ishan Kishan|2016-04-29|    1|
|Washington Sundar|2019-05-04|    1|
|   AB de Villiers|2020-10-10|    1|
|    SJ Srivastava|2010-03-24|    1|
|      S Sreesanth|2008-04-25|    1|
|        G Gambhir|2012-05-12|    1|
|       GJ Maxwell|2013-05-18|    1|
|         TL Suman|2013-04-30|    1|
|     AC Gilchrist|2013-04-14|    1|
|          SS Iyer|2015-04-26|    1|
|        AT Rayudu|2019-04-03|    1|
|   MJ McClenaghan|2018-04-22|    1|
|        DA Miller|2016-05-09|    1|
|         KL Rahul|2016-05-24|    1|
|       MA Agarwal|2018-05-12|    1|
|         R Dravid|2009-05-07|    1|
|         R Sharma|2011-05-04|    1|
|   Mohammed Shami|2017-04-17|    1|
|    Iqbal Abdulla|2015-05-04|    1|
+-----------------+----------+-----+
only showing top 20 rows



In [0]:
 # create SQL tables using dataframes
    
df.createOrReplaceTempView("matches")
spark.sql("""select * from matches""")

df1.createOrReplaceTempView("balls")
spark.sql("""select * from balls""")



Out[75]: DataFrame[id: string, inning: string, over: string, ball: string, batsman: string, non_striker: string, bowler: string, batsman_runs: int, extra_runs: int, total_runs: int, non_boundary: string, is_wicket: int, dismissal_kind: string, player_dismissed: string, fielder: string, extras_type: string, batting_team: string, bowling_team: string]

In [0]:
#Which player has taken most number of catches?

spark.sql(
"""
select fielder,count(*) from balls
where dismissal_kind == 'caught'
group by fielder
order by count(1) desc
"""
).show()

+--------------+--------+
|       fielder|count(1)|
+--------------+--------+
|    KD Karthik|     118|
|      MS Dhoni|     113|
|AB de Villiers|     103|
|      SK Raina|      99|
|     RG Sharma|      88|
|    RV Uthappa|      87|
|    KA Pollard|      84|
|       V Kohli|      76|
|      S Dhawan|      73|
|     MK Pandey|      70|
|      PA Patel|      69|
|     DA Warner|      66|
|       NV Ojha|      65|
|      DJ Bravo|      64|
|     RA Jadeja|      63|
|       WP Saha|      62|
|     AM Rahane|      58|
|     AT Rayudu|      56|
|  F du Plessis|      54|
|     SV Samson|      52|
+--------------+--------+
only showing top 20 rows



In [0]:
#Which team has best winning percentage in IPL?

matches=spark.sql(
"""

select matches_played, count(*)
from ((select team1 as matches_played from matches) union all
      (select team2 from matches)
     ) matches
group by matches_played
""")


winnerss=spark.sql(
"""
select winner,count(winner) from matches
group by winner
""")


In [0]:
#Which player has been run-out most number of times?

spark.sql(
"""
select player_dismissed,count(*) from balls
where dismissal_kind == 'run out'
group by player_dismissed
order by count(1) desc
"""
).show()

+----------------+--------+
|player_dismissed|count(1)|
+----------------+--------+
|       G Gambhir|      16|
|        S Dhawan|      15|
|        SK Raina|      13|
|       AT Rayudu|      13|
|  AB de Villiers|      12|
|      KD Karthik|      12|
|         M Vijay|      12|
| Y Venugopal Rao|      11|
|       YK Pathan|      11|
|        PA Patel|      10|
|       RG Sharma|      10|
|        A Mishra|      10|
|         P Kumar|      10|
|       IK Pathan|       9|
|        MS Dhoni|       9|
|      RV Uthappa|       9|
|       SR Watson|       9|
|    F du Plessis|       9|
|        AM Nayar|       8|
|DPMD Jayawardene|       8|
+----------------+--------+
only showing top 20 rows



In [0]:
# Which batsman has been hit wicket most number of times?

spark.sql(
"""
select batsman,count(*) from balls
where dismissal_kind == 'hit wicket'
group by batsman
order by count(1) desc
"""
).show()

+-------------+--------+
|      batsman|count(1)|
+-------------+--------+
|  SA Asnodkar|       1|
|     MA Khote|       1|
|Misbah-ul-Haq|       1|
|    RA Jadeja|       1|
|    SS Tiwary|       1|
| Yuvraj Singh|       1|
|    DA Warner|       1|
|     DJ Hooda|       1|
|   SP Jackson|       1|
|      R Parag|       1|
|    HH Pandya|       1|
|  Rashid Khan|       1|
+-------------+--------+



In [0]:
#Which player has taken most number of caught and bowled?

spark.sql(
"""
select bowler,count(*) from balls
where dismissal_kind == 'caught and bowled'
group by bowler
order by count(1) desc
"""
).show()

+---------------+--------+
|         bowler|count(1)|
+---------------+--------+
|       DJ Bravo|      11|
|Harbhajan Singh|      11|
|      SP Narine|       7|
|     SL Malinga|       6|
|        S Gopal|       6|
|     KA Pollard|       6|
|      PP Chawla|       6|
|      RA Jadeja|       5|
|  Kuldeep Yadav|       5|
|     JD Unadkat|       5|
|        B Kumar|       5|
|       S Nadeem|       5|
|    Imran Tahir|       5|
|       SK Warne|       5|
|     A Chandila|       5|
|    DS Kulkarni|       4|
|       R Ashwin|       4|
|       AR Patel|       4|
|  R Vinay Kumar|       4|
|      RS Bopara|       3|
+---------------+--------+
only showing top 20 rows



In [0]:
#Which player has most stumpings?

spark.sql(
"""
select fielder,count(*) from balls
where dismissal_kind == 'stumped'
group by fielder
order by count(1) desc
"""
).show()

+--------------+--------+
|       fielder|count(1)|
+--------------+--------+
|      MS Dhoni|      39|
|    RV Uthappa|      32|
|    KD Karthik|      30|
|       WP Saha|      20|
|      PA Patel|      16|
|  AC Gilchrist|      16|
|     Q de Kock|      12|
|       RR Pant|      11|
|       NV Ojha|      10|
| KC Sangakkara|       9|
|AB de Villiers|       8|
|    SP Goswami|       7|
|      MS Bisla|       7|
|     KM Jadhav|       7|
|   BB McCullum|       6|
|     SV Samson|       6|
|      KL Rahul|       5|
|       AP Tare|       5|
|     DH Yagnik|       5|
|   YV Takawale|       4|
+--------------+--------+
only showing top 20 rows



In [0]:
# Which team has won most number of matches in IPL?

spark.sql(
"""
select winner,count(winner) from matches
group by winner
order by count(winner) desc 
""").show()



+--------------------+-------------+
|              winner|count(winner)|
+--------------------+-------------+
|      Mumbai Indians|          120|
| Chennai Super Kings|          106|
|Kolkata Knight Ri...|           99|
|Royal Challengers...|           91|
|     Kings XI Punjab|           88|
|    Rajasthan Royals|           81|
|    Delhi Daredevils|           67|
| Sunrisers Hyderabad|           66|
|     Deccan Chargers|           29|
|      Delhi Capitals|           19|
|       Gujarat Lions|           13|
|       Pune Warriors|           12|
|Rising Pune Super...|           10|
|Kochi Tuskers Kerala|            6|
|Rising Pune Super...|            5|
|                  NA|            4|
+--------------------+-------------+



In [0]:
#Which team has best winning percentage in IPL?

matches=spark.sql(
"""

select matches_played, count(*)
from ((select team1 as matches_played from matches) union all
      (select team2 from matches)
     ) matches
group by matches_played
""")


winnerss=spark.sql(
"""
select winner,count(winner) from matches
group by winner
""")


winning   =   matches.join(winnerss,winnerss.winner==matches.matches_played,'inner')

winning_per   =   winning.withColumn("Winning Percentage", col('count(winner)')*100/col("count(1)")).sort('Winning Percentage',ascending=False)

win   =   winning_per[winning_per['count(winner)'] >= 20]

win.select('winner','Winning Percentage').show()


+--------------------+------------------+
|              winner|Winning Percentage|
+--------------------+------------------+
| Chennai Super Kings|59.550561797752806|
|      Mumbai Indians| 59.11330049261084|
| Sunrisers Hyderabad|53.225806451612904|
|Kolkata Knight Ri...|           51.5625|
|    Rajasthan Royals| 50.31055900621118|
|Royal Challengers...|46.666666666666664|
|     Kings XI Punjab| 46.31578947368421|
|    Delhi Daredevils| 41.61490683229814|
|     Deccan Chargers|38.666666666666664|
+--------------------+------------------+



In [0]:
#Which team won the match by biggest margin?

spark.sql(
"""
select id , winner , result_margin from matches
where result_margin != 'NA'
order by result_margin desc 
""").show(5)

+-------+-------------------+-------------+
|     id|             winner|result_margin|
+-------+-------------------+-------------+
| 419114|     Mumbai Indians|           98|
|1216510|    Kings XI Punjab|           97|
|1082599|   Delhi Daredevils|           97|
| 829753|Chennai Super Kings|           97|
| 729293|Chennai Super Kings|           93|
+-------+-------------------+-------------+
only showing top 5 rows



In [0]:
#Which player has most man of match awards in netural venue?

spark.sql(
"""
select player_of_match , count(player_of_match) from  matches
where neutral_venue = 1
group by player_of_match
order by count(player_of_match) desc 
""").show()

+----------------+----------------------+
| player_of_match|count(player_of_match)|
+----------------+----------------------+
|       JH Kallis|                     3|
|       JP Duminy|                     3|
|       YK Pathan|                     3|
|      GJ Maxwell|                     3|
|        DR Smith|                     2|
|   KC Sangakkara|                     2|
|       MK Pandey|                     2|
|       RG Sharma|                     2|
|  Sandeep Sharma|                     2|
|DPMD Jayawardene|                     2|
|  M Muralitharan|                     2|
|     LRPL Taylor|                     2|
|        SK Raina|                     2|
|    Yuvraj Singh|                     2|
|       ML Hayden|                     2|
|    SR Tendulkar|                     2|
|    AC Gilchrist|                     2|
|  AB de Villiers|                     2|
|      TM Dilshan|                     1|
|           B Lee|                     1|
+----------------+----------------

In [0]:
#Which team has won the toss most number of times in IPL?

spark.sql(
"""
select toss_winner , count(toss_winner) from matches
group by toss_winner
order by count(toss_winner) desc 
""").show()

+--------------------+------------------+
|         toss_winner|count(toss_winner)|
+--------------------+------------------+
|      Mumbai Indians|               106|
|Kolkata Knight Ri...|                98|
| Chennai Super Kings|                97|
|Royal Challengers...|                87|
|    Rajasthan Royals|                87|
|     Kings XI Punjab|                85|
|    Delhi Daredevils|                80|
| Sunrisers Hyderabad|                57|
|     Deccan Chargers|                43|
|      Delhi Capitals|                20|
|       Pune Warriors|                20|
|       Gujarat Lions|                15|
|Kochi Tuskers Kerala|                 8|
|Rising Pune Super...|                 7|
|Rising Pune Super...|                 6|
+--------------------+------------------+



In [0]:
# Which player has won most player_of_match awards?

spark.sql(
"""
select  player_of_match , count(player_of_match) from matches
group by player_of_match
order by count(player_of_match) desc 
""").show()

+---------------+----------------------+
|player_of_match|count(player_of_match)|
+---------------+----------------------+
| AB de Villiers|                    23|
|       CH Gayle|                    22|
|      RG Sharma|                    18|
|      DA Warner|                    17|
|       MS Dhoni|                    17|
|      SR Watson|                    16|
|      YK Pathan|                    16|
|       SK Raina|                    14|
|      G Gambhir|                    13|
|        V Kohli|                    13|
|     MEK Hussey|                    12|
|      AM Rahane|                    12|
|     KA Pollard|                    11|
|       DR Smith|                    11|
|     AD Russell|                    11|
|       A Mishra|                    11|
|       V Sehwag|                    11|
|      JH Kallis|                    10|
|      AT Rayudu|                    10|
|       SE Marsh|                     9|
+---------------+----------------------+
only showing top

In [0]:
# Query to find out who has officiated (as an umpire) the most number of matches in IPL

#Combing two similar columns by union and counting numbers on times they have officiated

spark.sql(
"""
select umpire, count(*)
from ( (select umpire1 as umpire from matches) union all (select umpire2 from matches) ) matches
group by umpire
order by count(*) desc ;
""").show()


+--------------------+--------+
|              umpire|count(1)|
+--------------------+--------+
|              S Ravi|     121|
|     HDPK Dharmasena|      94|
|        AK Chaudhary|      87|
|       C Shamshuddin|      82|
|           M Erasmus|      65|
|         Nitin Menon|      57|
|           CK Nandan|      57|
|          SJA Taufel|      55|
|           Asad Rauf|      51|
|         VA Kulkarni|      50|
|        BNJ Oxenford|      48|
|         CB Gaffaney|      47|
|           RJ Tucker|      46|
|         BR Doctrove|      42|
|         RE Koertzen|      41|
|           Aleem Dar|      38|
|            NJ Llong|      37|
|           BF Bowden|      37|
|KN Ananthapadmana...|      33|
|      RK Illingworth|      31|
+--------------------+--------+
only showing top 20 rows

